<a href="https://colab.research.google.com/github/IsuruKasun/Machine-Learning-Assignment--03/blob/main/assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from pandas import read_csv
from matplotlib.pyplot import bar, xlabel, ylabel, title
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# Import data from google drive 

from google.colab import drive
drive.mount('/content/drive')

directory_Path = '/content/drive/MyDrive/SYSC4415W23_A3_dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

file_path = '/content/drive/MyDrive/SYSC4415W23_A3_dataset/SYSC4415W23_A3_dataset/train/extracted_features.csv'

if os.path.exists(file_path):
    print('File exists!')
else:
    print('File does not exist.')

In [ ]:
# Load extracted features
train_extracted_features = pd.read_csv(directory_Path + '/SYSC4415W23_A3_dataset/train/extracted_features.csv')
test_extracted_features = pd.read_csv(directory_Path + '/SYSC4415W23_A3_dataset/test/extracted_features.csv')

train_extracted_features.head()
test_extracted_features.head()

In [ ]:
# Load labels
train_labels = pd.read_csv(directory_Path + '/SYSC4415W23_A3_dataset/train/labels.csv')
test_labels = pd.read_csv(directory_Path + '/SYSC4415W23_A3_dataset/test/labels.csv')

# train_labels.head()
# test_labels.head()

In [ ]:
# Load train_images data
#train_images = []

#train_images_path = directory_Path + '/train/images/*.png'
#for images in glob(train_images_path):
#   img = plt.imread(images)
#    train_images.append(img)

# print(train_images)


In [ ]:
# Load test_images data
#test_images = []

#test_images_path = directory_Path + '/test/images/*.png'
#for images in glob(test_images_path):
#    img = plt.imread(images)
#    test_images.append(img)

# print(test_images)

In [ ]:
# Dictionary to store sample and its associated pandas dataframe
# {key:value} = {sample id:pandas dataframe}
signals = {}

for sample_id in train_labels["sample_id"]:

  signals[sample_id] = pd.read_csv(directory_Path + "/SYSC4415W23_A3_dataset/train/signals/" + str(sample_id) + ".csv")

train_labels.shape
print(signals)

# for test data set

signals_test = {}

for sample_id in test_labels["sample_id"]:

  signals_test[sample_id] = pd.read_csv(directory_Path + "/SYSC4415W23_A3_dataset/test/signals/" + str(sample_id) + ".csv")

test_labels.shape
print(signals_test)


Marge data set 

In [ ]:

from pandas.core.reshape.merge import merge
merge_df_train = pd.merge(train_labels, train_extracted_features, on='sample_id') 
merge_df_train.head()

In [ ]:

merge_df_test = pd.merge(test_labels, test_extracted_features, on='sample_id') 
merge_df_test.head()


clearn data set

In [ ]:
merge_df_train.dropna(axis=1) # clearn the nan values
merge_df_train.fillna(merge_df_train.mean(), inplace=True)     
merge_df_train.head()

In [ ]:
merge_df_test.dropna(axis=1) # clearn the nan values
merge_df_test.fillna(merge_df_test.mean(), inplace=True)     
merge_df_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)

X_train_of_merge = selector.fit_transform(merge_df_train.iloc[:, 2:]) # make x features from data set merge_df_train
y_train_of_merge = merge_df_train["label"]
selected_features = merge_df_train.columns[selector.get_support(indices=True) + 2]
merge_df_train = pd.concat([merge_df_train[selected_features], y_train_of_merge], axis=1)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_variance_threshold.py:111: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


Use SVM to train the modle

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
#Train multiple models:

scaler.fit(X_train_of_merge)
svm_model = svm.SVC(kernel='linear')


In [ ]:
svm_model.fit(X_train_of_merge, y_train_of_merge)

SVC(kernel='linear')

In [ ]:
y_prediction = svm_model.predict(X_train_of_merge)
print(y_prediction)

[2 3 4 ... 0 4 1]


In [ ]:
accuracy = accuracy_score(y_train_of_merge, y_prediction)
print("Accuracy:", accuracy)

Accuracy: 0.999383096853794


Train data set Validataion 

In [ ]:
# divide tha data set 

from sklearn.model_selection import train_test_split


X_train_df, X_valid_df, y_train_df, y_valid_df = train_test_split(X_train_of_merge, y_train_of_merge, test_size=0.2, random_state=42)

In [ ]:
scaler.fit(X_train_df)
X_train_scaled = scaler.transform(X_train_df)
X_valid_scaled = scaler.transform(X_valid_df)


In [ ]:
svm_model.fit(X_train_scaled, y_train_df)

SVC(kernel='linear')